In [1]:
# import modules
import json
import pyodbc   
import requests 
import re

# this code is used if you have username and passpword on your SSMS
# with open('C:\\Users\\dgnpu\\Desktop\\DA-320\\DA_320_SQLServer\\data.json') as f:
#      data = json.load(f)
#      username = data['username']
#      password = data['password']


# make a connection to the database     
conn = pyodbc.connect("Driver={SQL Server};"
                     "Server=DESKTOP-J3LSHIL\MSSQLSERVER01;"
                     "Database=DA-320;")
                    #  "UID=" + username + ";"
                    #  "PWD=" + password)

# send request to the URL to get the data
r = requests.get('http://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2013&sort=desc&view=detailed')

#check the status of the request 
print(f"status:{r.status_code}")

# use to correct the 403 header
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11__5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

r = requests.get('https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2013&sort=desc&view=detailed', headers=headers)


# save the incoming data into these variables
titles = re.findall('class="title"><h3>(.*)<\/h3>', r.text)
release = re.findall('<span>(.*)<\/span>', r.text)
description = re.findall('class="summary">\n(.*)', r.text)
score = re.findall('<div class="metascore_w large movie positive">(.*)<\/div>\n\s+<\/a>\n\n\s+<\/div>\n\s+<span class="title numbered">', r.text)
image = re.findall('<a href="\S+ src="(.*) alt="(?!Metacritic logo")', r.text)

# use this code to sent the collected data to the database tables
cursor = conn.cursor()
sql = "INSERT INTO movies (movie_name, release_date, description, metascore, thumbnail) values (?,?,?,?,?)"
args = "Title", "Release date","Description", "Score", "Thumbnail"

# insert values to the database
arrayLength = range(len(titles))
for i in arrayLength:
    args = titles[i], release[i], description[i], score[i], image[i]
    print(args)
    cursor.execute(sql, args)
    cursor.commit()


 

status:403
('12 Years a Slave', 'October 18, 2013', '                        In the pre-Civil War United States, Solomon Northup, a free black man living in upstate New York, is abducted and sold into slavery.', '96', 'https://static.metacritic.com/images/products/movies/2/3910c2e8cfefeb21fcd6079451336f86-98.jpg"')
('Gravity', 'October 4, 2013', '                        Dr. Ryan Stone (Sandra Bullock) is a brilliant medical engineer on her first shuttle mission, with veteran astronaut Matt Kowalsky (George Clooney) in command of his last flight before retiring. But on a seemingly routine spacewalk, disaster strikes. The shuttle is destroyed, leaving Stone and Kowalsky completely alone - tethered to nothing but each other and spiraling out into the blackness. The deafening silence tells them they have lost any link to Earth - and any chance for rescue. As fear turns to panic, every gulp of air eats away at what little oxygen is left. But the only way home may be to go further out into t